# 第6章 主成分分析及Python计算

In [ ]:
#%cd "E:/应统电子书讲义\多元python
%run init.py

## 6.1 主成分分析的概念

### 6.1.1 主成分分析的提出

### 6.1.2 主成分的直观解释

$\begin{cases}
y_1=\cos \theta x_1+\sin \theta x_2 \\
y_2=-\sin \theta x_1+\cos \theta x_2
\end{cases}$

In [ ]:
x1=[147,171,175,159,155,152,158,154,164,168,166,159,164,177]  #身高
x2=[32,57,64,41,38,35,44,41,54,57,49,47,46,63]                #体重

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x1, x2); #绘制散点图
plt.xlabel('x1');plt.ylabel('x2');      

In [ ]:
from matplotlib.patches import Ellipse
fig=plt.figure();
ax=fig.add_subplot(111)
ell1=Ellipse(xy=(162,48),width=48,height=8,angle=48,facecolor='yellow',alpha=0.3) 
ax.add_patch(ell1) #绘制椭圆 
plt.scatter(x1, x2);plt.xlabel('x1');plt.ylabel('x2')    
plt.plot([146,178],[30,66]);plt.plot([162,166],[54,47]); #绘制线段
plt.text(178,66,'y1');plt.text(161,55,'y2');

## 6.2 主成分分析的性质

### 6.2.1 主成分的说明

In [ ]:
import pandas as pd
pd.set_option('display.precision',4)  #数据框输出精度
X=pd.DataFrame({'x1':x1,'x2':x2});#X  #构建数据框 X

In [ ]:
S=X.cov();S    #协方差阵

In [ ]:
R=X.corr();R  #相关系数阵

### 6.2.2 主成分的推导

In [ ]:
import numpy as np  #from numpy.linalg import svd
np.set_printoptions(4)
d1,u1,v1=np.linalg.svd(S) #协差阵的奇异值分解 S=UDV'
print('d1:\n',d1,'\n','u1:\n',u1,'\n','v1:\n',v1)

In [ ]:
d2,u2,v2=np.linalg.svd(R)  #相关阵的奇异值分解 R=UDV'
print('d1:\n',d2,'\n','u1:\n',u2,'\n','v1:\n',v2)

### 6.2.3 主成分的计算

1.主成分方差

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2).fit(X)         #拟合主成分
Vi=pca.explained_variance_;Vi             #主成分方差
#pd.DataFrame(pca.explained_variance_,index=['PC1','PC2'])   #主成分方差

2.主成分系数

In [ ]:
#DataFrame(pca.components_,columns=['PC1','PC2'])        #主成分负荷
pca.components_     #主成分负荷

3.主成分得分

In [ ]:
scores=pd.DataFrame(pca.fit_transform(X),columns=['PC1','PC2']);
scores #主成分得分
#scores=pca.fit_transform(X);scores #主成分得分

In [ ]:
scores.plot('PC1','PC2',kind='scatter')           #绘制主成分得分散点图
plt.axhline(y=0,ls=":");plt.axvline(x=0,ls=":"); #添加水平和垂直直线

In [ ]:
scores.corr().round(4)                  #主成分得分相关阵

In [ ]:
scores.cov().round(4)                    #主成分得分协方差阵

4.方差贡献率

In [ ]:
Wi=pca.explained_variance_ratio_;Wi   #方差贡献率 

In [ ]:
Wi.sum()             #方差累计贡献率

## 6.3 主成分分析步骤
### 6.3.1 计算过程

In [ ]:
import numpy as np
def PCscores(X,m=2): #主成分评价函数
    from sklearn.decomposition import PCA
    Z=(X-X.mean())/X.std() #数据标准化
    p=Z.shape[1]
    pca = PCA(n_components=p).fit(Z)
    Vi=pca.explained_variance_;Vi
    Wi=pca.explained_variance_ratio_;Wi
    Vars=pd.DataFrame({'Variances':Vi});Vars  #,index=X.columns
    Vars.index=['Comp%d' %(i+1) for i in range(p)]
    Vars['Explained']=Wi*100;Vars
    Vars['Cumulative']=np.cumsum(Wi)*100;
    print("\n方差贡献:\n",round(Vars,4))
    Compi=['Comp%d' %(i+1) for i in range(m)]
    loadings=pd.DataFrame(pca.components_[:m].T,columns=Compi,index=X.columns);
    print("\n主成分负荷:\n",round(loadings,4))
    scores=pd.DataFrame(pca.fit_transform(Z)).iloc[:,:m];
    scores.index=X.index; scores.columns=Compi;scores
    scores['Comp']=scores.dot(Wi[:m]);scores
    scores['Rank']=scores.Comp.rank(ascending=False).astype(int);
    return scores #print('\n综合得分与排名:\n',round(scores,4))

### 6.3.2 实证分析

In [ ]:
d31=pd.read_excel('mvsData.xlsx','d31',index_col=0)
d31_pcs=PCscores(d31);print(d31_pcs)

In [ ]:
d31=pd.read_excel('mvsData.xlsx','d31',index_col=0); d31

In [ ]:
d31_pcs=PCscores(d31);
#print(d31_pcs)

In [ ]:
print(d31_pcs.sort_values('Rank'))

In [ ]:
import matplotlib.pyplot as plt            
plt.rcParams['font.sans-serif']=['SimHei'];  #中文黑体SimHei
plt.rcParams['axes.unicode_minus']=False; #正常显示图中正负号
def Scoreplot(Scores): #自定得分图绘制函数
    plt.plot(Scores.iloc[:,0],Scores.iloc[:,1],'*'); 
    plt.xlabel(Scores.columns[0]);plt.ylabel(Scores.columns[1])
    plt.axhline(y=0,ls=':');plt.axvline(x=0,ls=':')
    for i in range(len(Scores)):
        plt.text(Scores.iloc[i,0],Scores.iloc[i,1],Scores.index[i])

In [ ]:
Scoreplot(d31_pcs)

## 6.4 主成分分析注意事项

## 案例6 电信业发展的主成分分析

In [ ]:
Case6=pd.read_excel('mvsCase.xlsx','Case6',index_col=0); #Case6

In [ ]:
plt.figure(figsize=(9,5))
import scipy.cluster.hierarchy as sch
Z=(Case6-Case6.mean())/Case6.std() #数据标准化
D=sch.distance.pdist(Z)
H=sch.linkage(D,method='complete');
sch.dendrogram(H,labels=Case6.index); #绘制系统聚类图

In [ ]:
Case6_pcs=PCscores(Case6);Case6_pcs    #主成分分析

In [ ]:
Scoreplot(Case6_pcs)